# 0. 개요

- 시계열 데이터

# 1. 라이브러리 로드 및 데이터 불러오기

## 1.1 라이브러리 로드 및 메모리 감소 함수 정의

In [2]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime
import os

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def write_record(features, params):
    record = open("record model and features.txt", 'a')
    record.write("\n")
    record.write(str(datetime.datetime.now())+"\n")

    check = 0
    for _ in features:
        check += 1
        if check % 5 == 0:
            record.write("\n")
        record.write(_+"  ")
    record.write("\n")
    for i  in params.items():
        record.write(str(i) + "\n")

    record.write('--------------------------------\n')
    record.close()

## 1.2 데이터 불러오기 및 pd.melt를 활용해 데이터 정렬

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

In [3]:
train = pd.read_csv('inputs/sales_train_validation.csv')
train = pd.melt(train, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='target')

test = pd.read_csv('inputs/sample_submission.csv')
test2 = test[30490:]

test = test[:30490]
test = pd.melt(test, id_vars=['id'], var_name='d', value_name='target')
for i in range(1, 29):
    test = test.replace({f'F{i}': f'd_{1913+i}'})

test[['cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'tmp']] = pd.DataFrame(test['id'].str.split('_').tolist())
del test['tmp']
test['store_id'] = test['state_id'] + '_' + test['store_id']
test['dept_id'] = test['cat_id'] + '_' + test['dept_id']
test['item_id'] = test['dept_id'] + '_' + test['item_id']

test = test[train.columns]

calendar = pd.read_csv('inputs/calendar.csv')

sell_prices = pd.read_csv('inputs/sell_prices.csv')

In [ ]:
test.tail()

# 2. 데이터 탐색

## 2.1 train

In [ ]:
train.head(2)

In [ ]:
# 아웃라이어를 제거해서 boxplot 더 잘보이게끔
train200 = train[train['target'] < 10]

### 2.1.1 target

In [ ]:
pd.set_option('display.max_rows', 419)
pd.DataFrame(train['target'].value_counts())

In [ ]:
plt.figure(figsize=(10,7))
display(
    train[train['target'] == 763],
    train[train['id'] == 'FOODS_3_090_CA_3_validation'],
    sns.distplot(train[train['id'] == 'FOODS_3_090_CA_3_validation']['target'])    
)

In [ ]:
plt.figure(figsize=(10,7))
display(
    train[train['target'] == 370],
    train[train['id'] == 'FOODS_3_318_CA_3_validation'],
    sns.distplot(train[train['id'] == 'FOODS_3_318_CA_3_validation']['target'])    
)

In [ ]:
train[train['target'] == 763]

In [ ]:
# 시간 오래걸림

plt.figure(figsize=(10,7))
# sns.distplot(train['target'])

### 2.1.2 id

In [ ]:
train.groupby('id')['target'].mean()

### 2.1.3 item_id

In [ ]:
train.groupby('item_id')['target'].mean()

In [ ]:
train['item_id'].value_counts()

### 2.1.4 dept_id

In [ ]:
train.groupby('dept_id')['target'].mean()

In [ ]:
train['dept_id'].value_counts()

In [ ]:
plt.figure(figsize=(10,7))
sns.boxplot(train['dept_id'], train200['target'])

### 2.1.5 cat_id

In [ ]:
train.groupby('cat_id')['target'].mean()

In [ ]:
train['cat_id'].value_counts()

### 2.1.6 store_id

In [ ]:
train.groupby('store_id')['target'].mean()

In [ ]:
train['store_id'].value_counts()

store_id 를 기준으로 10가지 모델을 만들어봐도 좋을 듯 싶다.  
우선 검증을 해야함. 각각이 많이 다른지부터

### 2.1.7 state_id

In [ ]:
train.groupby('state_id')['target'].mean()

In [ ]:
train['state_id'].value_counts()

### 2.1.8 d

In [ ]:
train.groupby('d')['target'].mean()

## 2.2 test

In [ ]:
test.head(2)

## 2.3 calendar

- 대부분 date 데이터고, event 및 snap(정부보조금 적용되는 날)에 집중해보자.
- event_name_2가 너무 적다. 우선을 빼고 모델 만들어 볼 것임.
- 이벤트 유무(is_event) 컬럼 만들 예정

In [4]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
calendar = calendar.drop(['event_name_2', 'event_type_2'], axis=1)
calendar['is_event'] = calendar['event_name_1'].notna().astype('int8')
del calendar['wday']
del calendar['wm_yr_wk'] # 모든 주차에 인덱스 붙여놓음. 282개

In [ ]:
calendar['day'] = calendar['date'].astype('datetime64').dt.day
calendar['week'] = calendar['date'].astype('datetime64').dt.week


In [ ]:
calendar.head()

In [ ]:
train = train.merge(calendar, how='left')
test = test.merge(calendar, how='left')

In [ ]:
train.head()

In [ ]:
train['day'] = train['date'].astype('datetime64').dt.day
test['day'] = test['date'].astype('datetime64').dt.day

train['week'] = train['date'].astype('datetime64').dt.week
test['week'] = test['date'].astype('datetime64').dt.week

## 2.4 sell_prices

In [ ]:
sell_prices.info()

In [ ]:
sell_prices.nunique()

In [ ]:
for i in sell_prices['wm_yr_wk'].unique():
    print(i)

In [ ]:
sell_prices.groupby('wm_yr_wk')['sell_price'].mean().index

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,7))
sns.lineplot(range(len(sell_prices.groupby('wm_yr_wk')['sell_price'].mean())), list(sell_prices.groupby('wm_yr_wk')['sell_price'].mean()))

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

In [1]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime
import os

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def write_record(features, params):
    record = open("record model and features.txt", 'a')
    record.write("\n")
    record.write(str(datetime.datetime.now())+"\n")

    check = 0
    for _ in features:
        check += 1
        if check % 5 == 0:
            record.write("\n")
        record.write(_+"  ")
    record.write("\n")
    for i  in params.items():
        record.write(str(i) + "\n")

    record.write('--------------------------------\n')
    record.close()

In [2]:
train = pd.read_csv('inputs/sales_train_validation.csv')
train = pd.melt(train, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='sales')
train = reduce_mem_usage(train)

Mem. usage decreased to 3226.27 Mb (9.4% reduction)


In [3]:
test = pd.read_csv('inputs/sample_submission.csv')
test = test[:30490]
test = pd.melt(test, id_vars=['id'], var_name='d', value_name='sales')
for i in range(1, 29):
    test = test.replace({f'F{i}': f'd_{1913+i}'})

test[['cat_id', 'dept_id', 'item_id', 'state_id', 'store_id', 'tmp']] = pd.DataFrame(test['id'].str.split('_').tolist())
del test['tmp']
test['store_id'] = test['state_id'] + '_' + test['store_id']
test['dept_id'] = test['cat_id'] + '_' + test['dept_id']
test['item_id'] = test['dept_id'] + '_' + test['item_id']

test = test[train.columns]
test = reduce_mem_usage(test)

Mem. usage decreased to 46.41 Mb (10.9% reduction)


## 3.2 calendar (date 데이터)

In [4]:
calendar = pd.read_csv('inputs/calendar.csv')

calendar = calendar.drop(['event_name_2', 'event_type_2'], axis=1)
calendar['is_event'] = calendar['event_name_1'].notna().astype('int8')
del calendar['wday']  # weekday랑 똑같은 컬럼.
calendar['day'] = calendar['date'].astype('datetime64').dt.day
calendar['week'] = calendar['date'].astype('datetime64').dt.week

In [5]:
train = train.merge(calendar, how='left')
test = test.merge(calendar, how='left')

## 3.3 sell_prices

In [6]:
sell_prices = pd.read_csv('inputs/sell_prices.csv')

In [7]:
train = train.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
test = test.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [8]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,0,29,4,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,0,29,4,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,0,29,4,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,0,29,4,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,0,0,0,0,29,4,NaN


## 3.4 라벨인코딩

In [ ]:
all_df = pd.concat([train, test])
all_df['revenue'] = all_df['sales'] * all_df['sell_price']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in all_df.columns[all_df.dtypes == 'object']:
    if i == 'id' or i == 'date':
        continue
    all_df[i] = le.fit_transform(list(all_df[i]))



In [ ]:
all_df = reduce_mem_usage(all_df)

In [ ]:
all_df.head()

## 3.5 lag 데이터 제작

In [ ]:
for i in range(28, 35):
    all_df[f'lag_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(i))



In [ ]:
all_df = reduce_mem_usage(all_df)

In [ ]:
for i in range(28, 35):
    all_df[f'revenue_lag_t{i}'] = all_df.groupby(['id'])['revenue'].transform(lambda x: x.shift(i))


In [ ]:
all_df = reduce_mem_usage(all_df)

In [ ]:
all_df.columns

In [ ]:
import pickle

def make_sales_lag(all_df, lag_day):
    for i in range(lag_day, lag_day + 14):
        all_df[f'lag_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(i))
        if i % 7 == 0:
            all_df = reduce_mem_usage(all_df)
    
    weeks = [7, 28, 56, 112, 168] # 7 30 60 120 180
    
    for i in weeks:
        all_df[f'rolling_max_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(lag_day).rolling(i).max())
        all_df[f'rolling_min_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(lag_day).rolling(i).min())
        all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(lag_day).rolling(i).mean())
        all_df[f'rolling_std_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(lag_day).rolling(i).std())
    
    all_df = reduce_mem_usage(all_df)
    with open(f'inputs/lag_rolling_{lag_day}.pickle', 'wb') as f:
        pickle.dump(all_df, f, protocol=4)

# pickle 저장하려고 return 그냥 없앴음
#     return all_df

In [ ]:
make_sales_lag(all_df, 1)

In [ ]:
all_df = make_target_lag(all_df, 1)

In [ ]:
# for i in range(1, 29):
#     all_df[f'lag_t{i}'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(i)).fillna(-i)
    
#     if i % 10 == 7:
#         all_df = reduce_mem_usage(all_df)

In [ ]:


# all_df['lag_t28'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(28))
# all_df['lag_t29'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(29))
# all_df['lag_t30'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(30))

# # 새롭게 만들 거
# all_df['lag_t24'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(24))
# all_df['lag_t25'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(25))
# all_df['lag_t26'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(26))
# all_df['lag_t27'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(27))

## 3.6 이동평균 피처

In [ ]:
all_df.head()

In [ ]:
# 모델을 shift를 1한거부터 28한거까지 28개 만들라고 한다.

weeks = [7, 28, 56, 84, 112, 168]
for i in weeks:
    all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(1).rolling(i).max())
    all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(1).rolling(i).min())
    all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(1).rolling(i).mean())
    all_df[f'rolling_std_t{i}'] = all_df.groupby(['id'])['target'].transform(lambda x: x.shift(1).rolling(i).std())
#     all_df = reduce_mem_usage(all_df)
print('finish!!')

In [ ]:
all_df.head()

In [ ]:
weeks = [7, 28, 56, 84, 112, 168] # 7, 30, 60, 90, 120, 180
for i in weeks:
    all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(i).max())
    all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(i).min())
    all_df[f'rolling_mean_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(i).mean())
    all_df[f'rolling_std_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(i).std())
#     all_df = reduce_mem_usage(all_df)
print('finish!!')


In [ ]:
all_df = reduce_mem_usage(all_df)

In [ ]:
all_df.head()

In [ ]:
all_df[(all_df.week == 17) & (all_df.year == 2015)]

### 연도별 판매량 그래프 분석중

- 2016년 4월을 맞춰야 해서, 2015년, 2014년, 2013년을 보고 있는데, 2015년의 그래프가 2014, 2013의 추이에 비해 다른 거 같다.

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(all_df[(all_df.week == 17) & (all_df.year == 2013)].groupby('item_id')['target'].mean()[:50])

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(all_df[(all_df.week == 17) & (all_df.year == 2014)].groupby('item_id')['target'].mean()[:50])

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(all_df[(all_df.week == 17) & (all_df.year == 2015)].groupby('item_id')['target'].mean()[:50])

In [ ]:
all_df[all_df.week == 17].groupby('year')['target'].mean()

In [ ]:
all_df[all_df.week == 18].groupby('year')['target'].mean()

In [ ]:
all_df[all_df.week == 19].groupby('year')['target'].mean()

In [ ]:
all_df[all_df.week == 20].groupby('year')['target'].mean()

## 3.7 price 통계량 피쳐

In [ ]:
for i in range(28, 35):
        all_df[f'price_lag_t{i}'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(i))

In [ ]:
all_df['rolling_price_std_t14'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(14).std())


In [ ]:
all_df['lag_price_t1'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))

all_df['price_change_t1'] = (all_df['lag_price_t1'] - all_df['sell_price']) / (all_df['lag_price_t1'])

all_df['rolling_price_max_t365'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t365'] = (all_df['rolling_price_max_t365'] - all_df['sell_price']) / (all_df['rolling_price_max_t365'])

all_df['rolling_price_std_t7'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())

# 새롭게 만들거
# all_df['rolling_price_std_t32'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(32).std())

all_df['rolling_price_std_t28'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(28).std())
# all_df = reduce_mem_usage(all_df)


all_df = all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], axis = 1)

In [ ]:
all_df = reduce_mem_usage(all_df)

In [ ]:
all_df.head()

In [ ]:
import pickle

with open('inputs/all_df6.pickle', 'wb') as f:
    pickle.dump(all_df, f, protocol=4)


In [ ]:
import pickle


with open('inputs/all_df.pickle', 'rb') as f:
    all_df = pickle.load(f)


# 4. 모델 제작

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
# rf.fit(train2, np.log(train['target'] + 1))
# result = rf.predict(test2)

In [ ]:
# from lightgbm import LGBMRegressor
# # lgb = LGBMRegressor(num_leaves=2000, colsample_bytree=0.6, subsample=0.6, n_estimators=600, learning_rate=0.025, n_jobs=-1, device='gpu', max_bin = 63)
# lgb = LGBMRegressor(num_leaves=20, colsample_bytree=0.6, subsample=0.6, n_estimators=60, learning_rate=0.02, n_jobs=-1, device='cpu')

# lgb.fit(train, target)
# result = lgb.predict(test)

## 4.1 KFold - LGBM 모델

In [ ]:
import lightgbm as lgb
import gc

features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'weekday', 'event_name_1', 'event_type_1',  
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30']

# 나중에 합칠 때 필요해서 test에 선언
test = all_df[len(train):]

train_set_X = all_df[:len(train)]
train_set_y = train_set_X['target']

train_set_X = train_set_X[features]

# 테스트 셋
test_set = all_df[len(train):]
test_set = test_set[features]

del all_df

gc.collect()

In [ ]:
pip uninstall pandas_profiling

In [ ]:
n_fold = 2
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test.shape[0])
y_oof = np.zeros(train.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []
eval_results = []

for fold_n, (train_index, valid_index) in enumerate(splits):
    
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        boosting_type = 'gbdt',
        num_leaves = 400,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 20,
        learning_rate = 0.01,
        n_jobs = -1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 50, verbose = True)
    eval_results.append(lgb.evals_result_)
    # 피쳐중요도 작성
    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    # validation predict
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)

    y_oof[valid_index] = y_pred_valid
    
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    
    print(f'val rmse score is {val_score}')
    
    mean_score.append(val_score)
    
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    del X_train, X_valid, y_train, y_valid

print('mean rmse score over folds is',np.mean(mean_score))
test['target'] = y_preds



In [ ]:
tmp1 = list(eval_results[0]['valid_0'].values())[0]

In [ ]:
tmp = list(eval_results[1]['valid_0'].values())[0]

In [ ]:
eval_results[0].values().values()

In [ ]:
eval_results[0]

In [ ]:
features = train_set_X.columns
params = lgb.get_params()

## 4.2 KFold - XGB 모델

In [ ]:
all_df = reduce_mem_usage(all_df)

test = all_df[len(train):]

features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'weekday', 'event_name_1', 'event_type_1',  
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price']



train_set_X = all_df[:len(train)]
train_set_y = train_set_X['target']

train_set_X = train_set_X[features]

# 테스트 셋
test_set = all_df[len(train):]
test_set = test_set[features]

del all_df

In [ ]:
train_set_X['sell_price'] = train_set_X['sell_price'].fillna(-1)

In [ ]:
train_set_X.head()

In [ ]:
import xgboost as xgb

n_fold = 5
folds = KFold(n_splits=5, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test.shape[0])
# y_oof = np.zeros(train.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

# dtest = xgb.DMatrix(data=test_set)


for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    
    xgb_model = xgb.XGBRegressor(colsample_bytree = 0.8, learning_rate = 0.02,subsample=0.8,
                max_depth = 12, n_estimators = 4000, tree_method='gpu_hist')
    
    xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=50)
    
    # 피쳐중요도 작성
    feature_importances[f'fold_{fold_n + 1}'] = xgb_model.feature_importances_
    
    # validation predict
    y_pred_valid = xgb_model.predict(X_valid)
    # y_oof[valid_index] = y_pred_valid
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')

    # test 값 예측
    y_preds += xgb_model.predict(test_set) / n_fold
    del X_train, X_valid, y_train, y_valid, y_pred_valid, val_score

print('mean rmse score over folds is',np.mean(mean_score))
test['target'] = y_preds

features = train_set_X.columns
params = xgb_model.get_params()

## 4.3 TimeSeriesSplit - LGBM 모델

In [ ]:
# n_fold = 10
# folds = TimeSeriesSplit(n_splits=n_fold)
# splits = folds.split(train_set_X, train_set_y)

# y_preds = np.zeros(test.shape[0])
# y_oof = np.zeros(train.shape[0])

# feature_importances = pd.DataFrame()
# feature_importances['feature'] = train_set_X.columns
# mean_score = []

# for fold_n, (train_index, valid_index) in enumerate(splits):
#     print('Fold:',fold_n+1)
    
#     X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
#     y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
#     lgb = LGBMRegressor(
#         num_leaves = 1000,
#         colsample_bytree = 0.8,
#         subsample = 0.8,
#         n_estimators = 2500,
#         learning_rate = 0.01,
#         n_jobs = -1,
#         device = 'cpu'
#     )
    
#     lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 50, verbose = False)

#     feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
#     y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
#     y_oof[valid_index] = y_pred_valid
    
#     val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    
#     print(f'val rmse score is {val_score}')
    
#     mean_score.append(val_score)
    
#     y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
#     del X_train, X_valid, y_train, y_valid

# print('mean rmse score over folds is',np.mean(mean_score))

# test['target'] = y_preds

## 4.4 feature_importance 확인

In [ ]:
feature_importances

# 5. 예측 및 제출

In [ ]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'target']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'target').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final.to_csv('submissions/submission.csv', index = False)

In [ ]:
for i in range(1,29):
    final['F'+str(i)] *= 1.04
    
submission.to_csv('sub.csv', index=False) 

In [ ]:
test.head()

In [ ]:
final.head()

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [ ]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

# 6. 모델 파라미터, 피처 기록 및 모델 저장하기

In [ ]:
write_record(features, params)

In [ ]:
import joblib
# save model
# joblib.dump(lgb, 'models/lgb1.pkl')
# load model
# lgb = joblib.load('models/lgb.pkl')

In [27]:
tmp1 = pd.read_csv('submissions/submission.csv')
tmp2 = pd.read_csv('submissions/submission1.csv')



In [17]:
tmp3 = tmp1[:30490].sort_values('id').reset_index(drop=True)

In [23]:
for i in range(1, 29):
    tmp3[f'F{i}'] = tmp3[f'F{i}'] * 0.5 + tmp4[f'F{i}'] * 0.5

In [18]:
tmp4 = tmp2[:30490]

In [22]:

tmp3.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,HOUSEHOLD_2_516_TX_2_validation,0.264382,0.227260,0.238415,0.251563,0.283494,0.321790,0.329201,0.268389,0.277989,0.259787,0.253154,0.280791,0.299535,0.307178,0.305109,0.259613,0.264304,0.255079,0.253656,0.325838,0.357291,0.262741,0.248800,0.269774,0.271202,0.288891,0.314515,0.312905
30486,HOUSEHOLD_2_516_TX_3_validation,0.123565,0.146030,0.160017,0.154190,0.170443,0.187848,0.189963,0.113738,0.163735,0.228379,0.267454,-0.211207,0.239072,0.442737,0.128051,0.449291,0.701380,0.420551,0.128821,0.220964,0.527878,0.224892,0.247345,0.228096,0.249551,0.259782,0.286912,0.233638
30487,HOUSEHOLD_2_516_WI_1_validation,0.054564,0.052851,0.046759,0.053853,0.075983,0.098401,0.098747,0.061978,0.071679,0.064387,0.063544,0.083152,0.109449,0.064891,0.067195,0.066569,0.067796,0.065404,0.108571,0.146021,0.144962,0.088349,0.094105,0.089260,0.093523,0.093794,0.131192,0.130971
30488,HOUSEHOLD_2_516_WI_2_validation,0.159665,0.137026,0.121786,0.121215,0.149286,0.164859,0.165062,0.122794,0.124304,0.116507,0.115516,0.136033,0.162729,0.118852,0.123027,0.108807,0.116938,0.116757,0.134753,0.158184,0.142374,0.098689,0.099440,0.089032,0.097805,0.120338,0.143473,0.145597
30489,HOUSEHOLD_2_516_WI_3_validation,0.055724,0.053526,0.050250,0.057430,0.082869,0.090550,0.100327,0.067746,0.077405,0.067664,0.065293,0.082118,0.098855,0.065623,0.057046,0.058732,0.062285,0.059799,0.079446,0.095920,0.094396,0.055668,0.064945,0.056837,0.064531,0.095609,0.108202,0.109065


In [21]:
tmp4.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,HOUSEHOLD_2_516_TX_2_validation,0.274609,0.254821,0.271785,0.256904,0.284230,0.365086,0.342923,0.268156,0.265087,0.254789,0.239026,0.262057,0.310986,0.264380,0.248688,0.243116,0.242963,0.244857,0.262264,0.344007,0.311437,0.238273,0.227813,0.238992,0.238626,0.258946,0.310066,0.279884
30486,HOUSEHOLD_2_516_TX_3_validation,0.164704,0.148694,0.164347,0.153357,0.171952,0.192675,0.166695,0.152499,0.151216,0.131934,0.143995,0.155688,0.190768,0.145409,0.150846,0.133590,0.134611,0.135197,0.148553,0.183003,0.172821,0.144301,0.140127,0.142686,0.144080,0.158547,0.191906,0.180816
30487,HOUSEHOLD_2_516_WI_1_validation,0.097810,0.093281,0.093208,0.094804,0.102164,0.126883,0.117037,0.104441,0.104559,0.100892,0.104523,0.119704,0.155542,0.115639,0.110027,0.107342,0.107523,0.111254,0.125078,0.149440,0.143049,0.106183,0.104043,0.103873,0.106900,0.133332,0.160926,0.153070
30488,HOUSEHOLD_2_516_WI_2_validation,0.058508,0.057269,0.057210,0.099556,0.102116,0.110251,0.108786,0.109087,0.112764,0.099079,0.112515,0.121708,0.129219,0.106326,0.116681,0.115251,0.115558,0.119019,0.130591,0.141278,0.136536,0.118030,0.116447,0.115218,0.101037,0.112318,0.121482,0.116816
30489,HOUSEHOLD_2_516_WI_3_validation,0.056479,0.053606,0.053495,0.053657,0.064931,0.068509,0.057046,0.105940,0.108187,0.098921,0.097635,0.111370,0.131200,0.098949,0.107640,0.104075,0.104352,0.106139,0.122871,0.144376,0.135247,0.108662,0.105231,0.104271,0.112338,0.130915,0.153850,0.142797


In [24]:

tmp3.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,HOUSEHOLD_2_516_TX_2_validation,0.269495,0.241041,0.255100,0.254234,0.283862,0.343438,0.336062,0.268272,0.271538,0.257288,0.246090,0.271424,0.305261,0.285779,0.276899,0.251365,0.253633,0.249968,0.257960,0.334922,0.334364,0.250507,0.238306,0.254383,0.254914,0.273918,0.312291,0.296395
30486,HOUSEHOLD_2_516_TX_3_validation,0.144135,0.147362,0.162182,0.153773,0.171197,0.190262,0.178329,0.133118,0.157476,0.180157,0.205725,-0.027759,0.214920,0.294073,0.139449,0.291441,0.417996,0.277874,0.138687,0.201983,0.350350,0.184596,0.193736,0.185391,0.196815,0.209165,0.239409,0.207227
30487,HOUSEHOLD_2_516_WI_1_validation,0.076187,0.073066,0.069983,0.074329,0.089073,0.112642,0.107892,0.083209,0.088119,0.082639,0.084033,0.101428,0.132496,0.090265,0.088611,0.086956,0.087659,0.088329,0.116825,0.147730,0.144005,0.097266,0.099074,0.096567,0.100211,0.113563,0.146059,0.142020
30488,HOUSEHOLD_2_516_WI_2_validation,0.109087,0.097148,0.089498,0.110385,0.125701,0.137555,0.136924,0.115940,0.118534,0.107793,0.114016,0.128870,0.145974,0.112589,0.119854,0.112029,0.116248,0.117888,0.132672,0.149731,0.139455,0.108360,0.107943,0.102125,0.099421,0.116328,0.132477,0.131206
30489,HOUSEHOLD_2_516_WI_3_validation,0.056102,0.053566,0.051872,0.055543,0.073900,0.079530,0.078686,0.086843,0.092796,0.083293,0.081464,0.096744,0.115027,0.082286,0.082343,0.081403,0.083318,0.082969,0.101159,0.120148,0.114821,0.082165,0.085088,0.080554,0.088435,0.113262,0.131026,0.125931


In [25]:
tmp2[30490:]

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,FOODS_1_001_CA_1_evaluation,0.909286,0.876865,0.868232,0.829987,1.079470,1.195339,1.051089,0.947232,0.964116,0.940157,0.948035,1.022156,1.263899,1.159840,0.923963,0.872116,0.907151,0.905133,1.014630,1.281301,1.179683,0.907707,0.837280,0.823530,0.833976,0.977012,1.194671,1.168449
30491,FOODS_1_001_CA_2_evaluation,1.125922,1.130784,1.055562,1.206748,1.269746,1.584282,1.400063,0.915172,0.986566,0.909642,0.901922,1.064120,1.506463,1.265644,0.977572,0.944617,0.955351,0.960568,1.113273,1.508013,1.568594,1.015881,0.944926,0.987723,0.774594,1.030730,1.362614,1.357858
30492,FOODS_1_001_CA_3_evaluation,1.087786,1.090677,1.033084,0.994754,1.088099,1.249545,1.121976,0.991365,1.124819,1.002626,0.981792,1.113109,1.316564,1.049997,1.049597,1.038145,1.063406,1.071524,1.157459,1.405349,1.417171,1.019436,1.007200,0.992844,0.999626,1.115108,1.186758,1.168864
30493,FOODS_1_001_CA_4_evaluation,0.399624,0.342678,0.331106,0.336598,0.402779,0.429038,0.441505,0.342629,0.388413,0.363998,0.360884,0.396833,0.420278,0.389779,0.345177,0.342588,0.384903,0.386537,0.433732,0.473272,0.444177,0.363981,0.356915,0.357877,0.373678,0.423823,0.487696,0.461303
30494,FOODS_1_001_TX_1_evaluation,0.193566,0.190363,0.182831,0.186084,0.189829,0.205109,0.204319,0.303272,0.321049,0.277842,0.283245,0.313116,0.342873,0.269834,0.260324,0.293366,0.307422,0.300666,0.325690,0.346597,0.315347,0.255004,0.253562,0.252858,0.264051,0.283310,0.315661,0.311888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_TX_2_evaluation,0.274609,0.254821,0.271785,0.256904,0.284230,0.365086,0.342923,0.268156,0.265087,0.254789,0.239026,0.262057,0.310986,0.264380,0.248688,0.243116,0.242963,0.244857,0.262264,0.344007,0.311437,0.238273,0.227813,0.238992,0.238626,0.258946,0.310066,0.279884
60976,HOUSEHOLD_2_516_TX_3_evaluation,0.164704,0.148694,0.164347,0.153357,0.171952,0.192675,0.166695,0.152499,0.151216,0.131934,0.143995,0.155688,0.190768,0.145409,0.150846,0.133590,0.134611,0.135197,0.148553,0.183003,0.172821,0.144301,0.140127,0.142686,0.144080,0.158547,0.191906,0.180816
60977,HOUSEHOLD_2_516_WI_1_evaluation,0.097810,0.093281,0.093208,0.094804,0.102164,0.126883,0.117037,0.104441,0.104559,0.100892,0.104523,0.119704,0.155542,0.115639,0.110027,0.107342,0.107523,0.111254,0.125078,0.149440,0.143049,0.106183,0.104043,0.103873,0.106900,0.133332,0.160926,0.153070
60978,HOUSEHOLD_2_516_WI_2_evaluation,0.058508,0.057269,0.057210,0.099556,0.102116,0.110251,0.108786,0.109087,0.112764,0.099079,0.112515,0.121708,0.129219,0.106326,0.116681,0.115251,0.115558,0.119019,0.130591,0.141278,0.136536,0.118030,0.116447,0.115218,0.101037,0.112318,0.121482,0.116816


In [30]:
tmp5 = pd.concat([tmp3, tmp2[30490:]])

In [31]:
tmp5.to_csv('sub.csv', index=False)